In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ax-platform

     |████████████████████████████████| 950 kB 5.3 MB/s 
     |████████████████████████████████| 347 kB 27.2 MB/s 
     |████████████████████████████████| 310 kB 39.0 MB/s 
  Created wheel for gpytorch: filename=gpytorch-1.6.0-py2.py3-none-any.whl size=509889 sha256=4163a9ce49c152b1b27fe6266bc9e1d4d28b1bcce22b4516f2aa78f2e7156db3
  Stored in directory: /root/.cache/pip/wheels/66/b5/89/34c06ad393a6feb72b4cdde46d0f1c667f3e2632960f9df109
Successfully built gpytorch


In [3]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

from ax.plot.slice import plot_slice
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render
from ax.utils.tutorials.cnn_utils import train, evaluate
from ax.utils.notebook.plotting import render, init_notebook_plotting

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

warnings.filterwarnings('ignore')
init_notebook_plotting()

Output hidden; open in https://colab.research.google.com to view.

In [4]:
torch.manual_seed(0)
dtype = torch.float
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

b_synthetic_calls_path = '/content/drive/MyDrive/Progetto Stage/data/binom_synthetic_calls.csv'
b_synthetic_puts_path = '/content/drive/MyDrive/Progetto Stage/data/binom_synthetic_puts.csv'
t_synthetic_calls_path = '/content/drive/MyDrive/Progetto Stage/data/trinomial_synthetic_calls.csv'
t_synthetic_puts_path = '/content/drive/MyDrive/Progetto Stage/data/trinomial_synthetic_puts.csv'

In [5]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    
    return df

In [6]:
class OptDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

  def __len__(self):
    return len(self.X)

In [7]:
def preprocessing(df:pd.DataFrame):
  df = pd.get_dummies(df, prefix='', prefix_sep='')
  input_sc = StandardScaler()
  output_sc = StandardScaler()
  input_data = input_sc.fit_transform(df.drop('Option Price', axis=1))
  output_data = output_sc.fit_transform(df['Option Price'].values.reshape(-1, 1))

  return input_data, output_data

In [8]:
def load_data():
  b_synthetic_calls = pd.read_csv(b_synthetic_calls_path, index_col=0)
  b_synthetic_puts = pd.read_csv(b_synthetic_puts_path, index_col=0)
  t_synthetic_calls = pd.read_csv(t_synthetic_calls_path, index_col=0)
  t_synthetic_puts = pd.read_csv(t_synthetic_puts_path, index_col=0)

  b_synthetic_calls = reduce_mem_usage(b_synthetic_calls)
  b_synthetic_puts = reduce_mem_usage(b_synthetic_puts)
  
  t_synthetic_calls = reduce_mem_usage(t_synthetic_calls)
  t_synthetic_puts = reduce_mem_usage(t_synthetic_puts)
  
  b_synthetic_calls = shuffle(b_synthetic_calls, random_state=0)
  b_synthetic_puts = shuffle(b_synthetic_puts, random_state=0)
  t_synthetic_calls = shuffle(t_synthetic_calls, random_state=0)
  t_synthetic_puts = shuffle(t_synthetic_puts, random_state=0)

  synthetic_options = pd.concat([
                                 b_synthetic_calls[0:len(b_synthetic_calls) // 2], 
                                 b_synthetic_puts[0:len(b_synthetic_puts) // 2],
                                 t_synthetic_puts[0:len(t_synthetic_puts) // 2],
                                 t_synthetic_calls[0:len(t_synthetic_calls) // 2]],
                                 ignore_index=True)
  synthetic_options = shuffle(synthetic_options, random_state=0)
  
  input_data, output_data = preprocessing(synthetic_options)

  train_size = 0.8

  last_train_idx = int(np.round(len(input_data) * train_size))

  X_train = Variable(torch.Tensor(input_data[0:last_train_idx]))
  X_test = Variable(torch.Tensor(input_data[last_train_idx:]))

  y_train = Variable(torch.Tensor(output_data[0:last_train_idx]))
  y_test = Variable(torch.Tensor(output_data[last_train_idx:]))

  return OptDataset(X_train, y_train), OptDataset(X_test, y_test)

# Configurable Model

In [9]:
class ResBlock(nn.Module):

  def __init__(self, module):
    super(ResBlock, self).__init__()
    self.module = module

  def forward(self, x):
    return self.module(x) + x

In [10]:
class HiddenLayer(nn.Module):

  def __init__(self, layer_size, act_fn):
      super(HiddenLayer, self).__init__()
      
      if act_fn == 'ReLU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.ReLU())
      elif act_fn == 'LeakyReLU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.LeakyReLU())
      elif act_fn == 'ELU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.ELU())
    
  def forward(self, x):
    return self.layer(x)

In [11]:
class Net(nn.Module):

  def __init__(self, input_size, output_size, hidden_size, num_layers, act_fn):
    super(Net, self).__init__()
    self.input_size = input_size
    self.output_size = output_size
    self.hidden_size = hidden_size

    if act_fn == 'ReLU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.ReLU())
    elif act_fn == 'LeakyReLU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.LeakyReLU())
    elif act_fn == 'ELU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.ELU())

    self.hidden_layers_list = []

    for i in range(num_layers // 2):
      self.hidden_layers_list.append(
          ResBlock(
            nn.Sequential(
                HiddenLayer(self.hidden_size, act_fn),
                HiddenLayer(self.hidden_size, act_fn)
            )
        )
      )

    self.hidden_layers = nn.Sequential(*self.hidden_layers_list)

    self.net = nn.Sequential(
        self.initial_layer,
        self.hidden_layers,
        nn.Linear(self.hidden_size, self.output_size)
    )
  
  def forward(self, x):
    return self.net(x)

In [12]:
@torch.no_grad()
def init_xuniform(m):
  if isinstance(m, nn.Linear):
    torch.nn.init.xavier_uniform_(m.weight)
    m.bias.data.fill_(0.01)

In [13]:
input_size = 7
output_size = 1
number_of_samples = 30

In [14]:
def net_train(net, train_loader, parameters, dtype, device):
  net.to(dtype=dtype, device=device)

  # Define loss and optimizer
  criterion = nn.MSELoss()
  optimizer = optim.Adam(net.parameters(),
                        lr=parameters.get("lr", 0.001))

  num_epochs = 50
  # Train Network
  for _ in range(num_epochs):
      for inputs, labels in train_loader:
          # move data to proper dtype and device
          inputs = inputs.to(dtype=dtype, device=device)
          labels = labels.to(device=device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

  return net

In [15]:
def train_evaluate(parameterization):
  
    train_loader = torch.utils.data.DataLoader(
        trainset,
        batch_size=int(parameterization.get("batch_size")),
        shuffle=True,
        num_workers=2)
    
    test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=int(parameterization.get("batch_size")),
        shuffle=True,
        num_workers=2)
    
    untrained_net = Net(
        input_size, 
        output_size, 
        parameterization.get("hidden_size"), 
        parameterization.get("num_layers"), 
        parameterization.get("act_fn"))
    
    untrained_net.apply(init_xuniform)
    
    # train
    trained_net = net_train(net=untrained_net, train_loader=train_loader, 
                            parameters=parameterization, dtype=dtype, device=device)
    
    loss_fn = nn.MSELoss()
    mean_loss, sem = test_accuracy(trained_net, loss_fn, test_loader)
    
    # return the accuracy of the model as it was trained in this run
    return {'mse': (mean_loss, sem)}

In [16]:
def test_accuracy(net, loss_fn, testloader):

    losses = []
    with torch.no_grad():
        for data in testloader:
            X, y = data
            X, y = X.to(device), y.to(device)
            out = net(X)
            loss = loss_fn(out, y.to(device))
            losses.append(loss.cpu().item())


    losses = np.array(losses)

    return losses.mean(), losses.std() / np.sqrt(len(losses))

In [17]:
trainset, testset = load_data()

In [18]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-5, 0.1], "log_scale": True, "value_type":"float"},
        {"name": "batch_size", "type": "range", "bounds": [512, 2048], "value_type":"int"},
        {"name": "act_fn", "type": "choice", "values": ["ReLU", "LeakyReLU", "ELU"], "value_type":"str"},
        {"name": "num_layers", "type": "choice", "values": [4, 6, 8], "value_type":"int"},
        {"name": "hidden_size", "type": "choice", "values": [400, 600, 800], "value_type":"int"},        
    ],
    evaluation_function=train_evaluate,
    objective_name='mse',
    minimize=True,
    total_trials=number_of_samples,
    random_seed=0
)

[INFO 04-21 08:18:52] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='lr', parameter_type=FLOAT, range=[1e-05, 0.1], log_scale=True), RangeParameter(name='batch_size', parameter_type=INT, range=[512, 2048]), ChoiceParameter(name='act_fn', parameter_type=STRING, values=['ReLU', 'LeakyReLU', 'ELU'], is_ordered=False, sort_values=False), ChoiceParameter(name='num_layers', parameter_type=INT, values=[4, 6, 8], is_ordered=True, sort_values=True), ChoiceParameter(name='hidden_size', parameter_type=INT, values=[400, 600, 800], is_ordered=True, sort_values=True)], parameter_constraints=[]).
[INFO 04-21 08:18:52] ax.modelbridge.dispatch_utils: Using Bayesian optimization since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 04-21 08:18:52] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 10 trials, GP

In [19]:
best_parameters

{'act_fn': 'LeakyReLU',
 'batch_size': 1139,
 'hidden_size': 600,
 'lr': 0.00013699522195324395,
 'num_layers': 4}

In [20]:
data = experiment.fetch_data()
df = data.df
df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,mse,5.219214e-04,1.001993e-05,0
1,1_0,mse,1.727394e+04,7.613053e+01,1
2,2_0,mse,5.742531e-04,9.223808e-06,2
3,3_0,mse,5.429757e-04,1.021231e-05,3
4,4_0,mse,5.052491e-04,8.591915e-06,4
5,5_0,mse,5.492758e-04,1.040220e-05,5
6,6_0,mse,7.513573e+09,7.976923e+07,6
7,7_0,mse,5.271295e-04,9.795044e-06,7
8,8_0,mse,7.715901e-04,1.175883e-05,8
9,9_0,mse,6.851884e+22,6.776471e+21,9


In [21]:
trials_df = pd.DataFrame({t.arm.name: t.arm.parameters for t in experiment.trials.values()}).T.reset_index().rename({'index':'arm_name'}, axis=1)
trials_df

,arm_name,lr,batch_size,num_layers,hidden_size,act_fn
0,0_0,0.000795,1422,4,400,ReLU
1,1_0,0.013596,1005,8,800,ELU
2,2_0,0.003969,2013,8,400,ReLU
3,3_0,0.000023,895,4,600,LeakyReLU
4,4_0,0.000051,1753,4,600,LeakyReLU
5,5_0,0.002769,579,8,800,ReLU
6,6_0,0.06173,1493,6,400,ELU
7,7_0,0.000114,1127,6,800,ReLU
8,8_0,0.000287,1940,6,600,ELU
9,9_0,0.04947,752,8,600,LeakyReLU


In [22]:
res_ = trials_df.set_index('arm_name').join(df.set_index('arm_name')).sort_values('mean')
res_

,lr,batch_size,num_layers,hidden_size,act_fn,metric_name,mean,sem,trial_index
arm_name,,,,,,,,,
26_0,0.000137,1139,4,600,LeakyReLU,mse,4.486556e-04,8.050248e-06,26
27_0,0.000139,1129,4,600,LeakyReLU,mse,4.689799e-04,7.989670e-06,27
28_0,0.00014,1137,4,600,LeakyReLU,mse,4.707465e-04,1.091911e-05,28
29_0,0.000141,1128,4,600,LeakyReLU,mse,4.717863e-04,9.918419e-06,29
25_0,0.00013,1144,4,600,LeakyReLU,mse,4.841775e-04,8.388749e-06,25
14_0,0.000218,1861,6,400,ReLU,mse,4.842584e-04,9.958723e-06,14
12_0,0.000182,1256,8,600,ReLU,mse,4.942191e-04,9.925542e-06,12
4_0,0.000051,1753,4,600,LeakyReLU,mse,5.052491e-04,8.591915e-06,4
17_0,0.000719,1512,6,600,ReLU,mse,5.068355e-04,9.546042e-06,17


In [23]:
res_.to_csv('/content/drive/MyDrive/Progetto Stage/results/bin-tri_bo_results.csv')